In [1]:
#!pip install chesslab

In [2]:
from sklearn.model_selection import train_test_split
from chesslab.utils import load_pkl
from chesslab.training_torch import fitting
import torch
import torch.nn as nn
import numpy as np

In [3]:
example=1
lr = 0.1
epochs=29
batch_size = 128
test_percent=0.1
path = 'D:/database/ccrl/'
name_data='ccrl_states_elo2.pkl'
name_labels='ccrl_results_elo2.pkl'
save_name='./tmp/test_elo.0'
num_workers=4
loss_fn=nn.CrossEntropyLoss(reduction='mean')
optim=torch.optim.SGD

In [4]:
download=False
if download:
    from chesslab.utils import download_7z
    path='./'
    file_id = '1MFHFz_rxNziYSeN-9ruwnRiYskd0_9ss'
    download_7z(file_id,path)

In [5]:
encoding_1={
    '.':torch.tensor([0 , 0, 0], dtype=torch.float),
    'p':torch.tensor([0 , 0, 1], dtype=torch.float),
    'P':torch.tensor([0 , 0,-1], dtype=torch.float),
    'b':torch.tensor([0 , 1, 0], dtype=torch.float),
    'B':torch.tensor([0 ,-1, 0], dtype=torch.float),
    'n':torch.tensor([1 , 0, 0], dtype=torch.float),
    'N':torch.tensor([-1, 0, 0], dtype=torch.float),
    'r':torch.tensor([0 , 1, 1], dtype=torch.float),
    'R':torch.tensor([0 ,-1,-1], dtype=torch.float),
    'q':torch.tensor([1 , 0, 1], dtype=torch.float),
    'Q':torch.tensor([-1, 0,-1], dtype=torch.float),
    'k':torch.tensor([1 , 1, 0], dtype=torch.float),
    'K':torch.tensor([-1,-1, 0], dtype=torch.float)
}

encoding_2={
    '.':torch.tensor([0,0,0,0],dtype=torch.float),
    'p':torch.tensor([1,0,0,0],dtype=torch.float),
    'P':torch.tensor([0,0,0,1],dtype=torch.float),
    'b':torch.tensor([0,1,0,0],dtype=torch.float),
    'B':torch.tensor([0,0,1,0],dtype=torch.float),
    'n':torch.tensor([1,1,0,0],dtype=torch.float),
    'N':torch.tensor([0,0,1,1],dtype=torch.float),
    'r':torch.tensor([1,0,1,0],dtype=torch.float),
    'R':torch.tensor([0,1,0,1],dtype=torch.float),
    'q':torch.tensor([1,0,0,1],dtype=torch.float),
    'Q':torch.tensor([0,1,1,0],dtype=torch.float),
    'k':torch.tensor([1,1,1,0],dtype=torch.float),
    'K':torch.tensor([0,1,1,1],dtype=torch.float)
}

In [6]:
class Model_1(nn.Module):

    def __init__(self):
        super().__init__()
        self.func_1=nn.ELU()
        self.func_2=nn.ELU()
        self.func_3=nn.ELU()
        self.func_4=nn.ELU()
        
        self.cnn_1 = nn.Conv2d(3, 32, kernel_size=7,padding=3)
        self.cnn_2 = nn.Conv2d(32, 64, kernel_size=5,padding=2)
        self.cnn_3 = nn.Conv2d(64, 128, kernel_size=3,padding=1)

        self.linear_1 = nn.Linear(8*8*128,256 )
        self.linear_2 = nn.Linear(256, 2)

    def forward(self, x ):
        out = self.cnn_1(x)
        out = self.func_1(out)
        out = self.cnn_2(out)
        out = self.func_2(out)
        out = self.cnn_3(out)
        out = self.func_3(out)
        out = out.reshape([x.size(0), -1])
        out = self.linear_1(out)
        out = self.func_4(out)
        out = self.linear_2(out)

        return out
    
class Model_2(nn.Module):

    def __init__(self):
        super().__init__()
        self.func_1=nn.ReLU()
        self.func_2=nn.ReLU()
        self.func_3=nn.ReLU()
        self.func_4=nn.ReLU()
        
        self.cnn_1 = nn.Conv2d(4, 32, kernel_size=7,padding=3)
        self.cnn_2 = nn.Conv2d(32, 64, kernel_size=5,padding=2)
        self.cnn_3 = nn.Conv2d(64, 128, kernel_size=3,padding=1)

        self.linear_1 = nn.Linear(8*8*128,256 )
        self.linear_2 = nn.Linear(256, 2)

    def forward(self, x ):
        out = self.cnn_1(x)
        out = self.func_1(out)
        out = self.cnn_2(out)
        out = self.func_2(out)
        out = self.cnn_3(out)
        out = self.func_3(out)
        out = out.reshape([x.size(0), -1])
        out = self.linear_1(out)
        out = self.func_4(out)
        out = self.linear_2(out)

        return out

In [7]:
np.random.seed(0)
torch.manual_seed(0)

x_data = load_pkl(path+name_data)[:1000000]
y_data = load_pkl(path+name_labels)[:1000000,1] #Nota: las etiquetas deben de ser enteros, no onehot

print(x_data.shape)
print(y_data.shape)

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size = test_percent, random_state = 0, shuffle = True)

del x_data
del y_data

(1000000, 64)
(1000000,)


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if example==1:
    model = Model_1().to(device)
    encoding=encoding_1
else:
    model = Model_2().to(device)
    encoding=encoding_2

print(device)
print(model)

cuda
Model_1(
  (func_1): ELU(alpha=1.0)
  (func_2): ELU(alpha=1.0)
  (func_3): ELU(alpha=1.0)
  (func_4): ELU(alpha=1.0)
  (cnn_1): Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (cnn_2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear_1): Linear(in_features=8192, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=2, bias=True)
)


In [9]:
fitting(epochs=epochs,
        x_train=x_train,
        y_train=y_train,
        x_test=x_test,
        y_test=y_test,
        device=device,
        model=model,
        optim=optim,
        batch_size=batch_size,
        lr=lr,
        loss_fn=loss_fn,
        save_name=save_name,
        encoding=encoding,
        num_workers=num_workers)

2021-10-24 15:45:53


Epoch: 01/29 | time: 55s = 0.9m | train loss: 0.5693 | train acc: 0.6908 | test loss: 0.5783 | test acc: 0.6912

Epoch: 02/29 | time: 54s = 0.9m | train loss: 0.5276 | train acc: 0.7224 | test loss: 0.5221 | test acc: 0.7236

Epoch: 03/29 | time: 55s = 0.9m | train loss: 0.5128 | train acc: 0.7330 | test loss: 0.5190 | test acc: 0.7290

Epoch: 04/29 | time: 54s = 0.9m | train loss: 0.4993 | train acc: 0.7419 | test loss: 0.5052 | test acc: 0.7411

...

Epoch: 25/29 | time: 61s = 1.0m | train loss: 0.0535 | train acc: 0.9809 | test loss: 0.2194 | test acc: 0.9380

Epoch: 26/29 | time: 62s = 1.0m | train loss: 0.0492 | train acc: 0.9826 | test loss: 0.2409 | test acc: 0.9376

Epoch: 27/29 | time: 62s = 1.0m | train loss: 0.0451 | train acc: 0.9841 | test loss: 0.2421 | test acc: 0.9418

Epoch: 28/29 | time: 64s = 1.1m | train loss: 0.0419 | train acc: 0.9855 | test loss: 0.2677 | test acc: 0.9354

Epoch: 29/29 | time: 61s = 1.0m | train loss: 0.0401 | train acc: 0.9863 | test loss: 0.2800 | test acc: 0.9389

In [11]:
fitting(epochs=1,
      x_train=x_train,
      y_train=y_train,
      x_test=x_test,
      y_test=y_test,
      device=device,
      model= model, 
      load_name='tmp/test_elo.0.29.pt',
      save_name=save_name,
      num_workers=num_workers)

2021-10-24 16:27:08


Epoch: 30/30 | time: 60s = 1.0m | train loss: 0.0388 | train acc: 0.9868 | test loss: 0.2530 | test acc: 0.9413